In [1]:
from google.cloud import speech
from google.cloud import storage
from google.oauth2 import service_account
import json
import base64
import requests
import time
import pandas as pd

In [2]:
credentials = service_account.Credentials.from_service_account_file("smart-dyspnea-ml-20e2f40d2865.json")

### Store in Google Cloud Storage

## mp3 audios

In [3]:
BUCKET_DESTINO = "smart-dyspnea-audios"

In [10]:
storage_client = storage.Client(credentials=credentials)

In [7]:
with open('./data/ml_data.json', 'r') as f:
    data = json.loads(f.read())

In [59]:
# guardamos en google cloud storage
for record in data: #[:20]:
    bucket = storage_client.bucket(BUCKET_DESTINO)
    blob = bucket.blob(record['id'])
    audio = base64.b64decode(record['audio']['data'].encode('utf-8'))
    blob.upload_from_string(audio)

In [291]:
aux = [d for d in data if "109d0932-6702-4390-9e4c-d01177e1fb5c" in d["id"]]

In [292]:
from IPython.display import Audio
Audio(base64.b64decode(aux[0]['audio']['data'].encode('utf-8')))

TypeError: string indices must be integers

## wav normalized audios

In [270]:
BUCKET_DESTINO = "smart-dyspnea-audios-normalized"

In [271]:
storage_client = storage.Client(credentials=credentials)

In [272]:
with open('./data/ml_data_normalized.json', 'r') as f:
    data = json.loads(f.read())

In [275]:
# guardamos en google cloud storage
for record in data: #[:20]:
    bucket = storage_client.bucket(BUCKET_DESTINO)
    blob = bucket.blob(record['id'])
    audio = base64.b64decode(record['audio'].encode('utf-8'))
    blob.upload_from_string(audio)

In [294]:
aux = [d for d in data if "sources/13-01-2021_12:17:09-2f78e3ed-599e-4b9d-b519-11a234970dbb" in d["id"]]

In [295]:
from IPython.display import Audio
Audio(base64.b64decode(aux[0]['audio'].encode('utf-8')))

### Process multiple audios

Info usada para llamar a speech to text:

- https://cloud.google.com/speech-to-text/docs/libraries
- https://googleapis.dev/python/speech/latest/speech_v1/types.html#google.cloud.speech_v1.types.RecognitionConfig
- https://cloud.google.com/speech-to-text/docs/context-strength
- https://cloud.google.com/speech-to-text/docs/class-tokens


In [235]:
from google.cloud import speech_v1p1beta1 as speech

In [236]:
speech_client = speech.SpeechClient(credentials=credentials)

In [241]:
#######
# Only one transcription
# choose one
id_starts = "sources/12-01-2021_18:38:42-4ee43891-ba37-41e2"
#id_starts = "sources/17-01-2021_16:44:38-52f93e1c-5f99-4e40"
#id_starts = "sources/31-12-2020_19:46:31-a803d8b4-075a-4722"

record = [d for d in data if d["id"].startswith(id_starts)][0]

In [242]:
audio_id = record['id']

gcs_uri = f"gs://{BUCKET_DESTINO}/{audio_id}"
audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    #encoding=speech.RecognitionConfig.AudioEncoding.MP3,
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    #sample_rate_hertz=48000,
    sample_rate_hertz=22500,
    language_code="es-ES",
    enable_speaker_diarization=True,
    speech_contexts=[speech.SpeechContext(phrases=[str(i) for i in range(1, 31)])],  #  "$OOV_CLASS_ALPHANUMERIC_SEQUENCE", OOV_CLASS_DIGIT_SEQUENCE
)

# Detects speech in the audio file
start = time.time()
response = speech_client.recognize(config=config, audio=audio)
end = time.time()
print(f"Time spent: {end-start}")

# TODO: this should be done instead working with trascription
# response.results[0].alternatives[0].words[0..N] and .start_time/.end_time (in micros), and .word (value)

for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

Time spent: 2.5061633586883545
Transcript: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Transcript: 


In [92]:
response.results[0].alternatives[0].words[1]

#######

start_time {
  nanos: 700000000
}
end_time {
  seconds: 1
}
word: "2"

In [276]:
results = {}

VALID_SEQUENCE = [str(i) for i in range(1, 31)]

for record in data:
    
    audio_id = record['id']
    
    gcs_uri = f"gs://{BUCKET_DESTINO}/{audio_id}"
    audio = speech.RecognitionAudio(uri=gcs_uri)
    
    config = speech.RecognitionConfig(
        #encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        #sample_rate_hertz=48000,
        sample_rate_hertz=22500,
        language_code="es-ES",
        enable_speaker_diarization=True,
        speech_contexts=[speech.SpeechContext(phrases=VALID_SEQUENCE)],  #  "$OOV_CLASS_ALPHANUMERIC_SEQUENCE", OOV_CLASS_DIGIT_SEQUENCE
        #speech_contexts=[speech.SpeechContext(phrases=["$OOV_CLASS_DIGIT_SEQUENCE"])],  #  "$OOV_CLASS_ALPHANUMERIC_SEQUENCE", OOV_CLASS_DIGIT_SEQUENCE
    )

    # Detects speech in the audio file
    start = time.time()
    response = speech_client.recognize(config=config, audio=audio)
    end = time.time()
    print(f"ID: {audio_id}. Time spent: {end-start}")
    
    # TODO: this should be done instead working with trascription
    # response.results[0].alternatives[0].words[0..N] and .start_time/.end_time (in micros), and .word (value)

    words = response.results[0].alternatives[0].words if response.results else None
    transcription = response.results[0].alternatives[0].transcript if response.results else None
    # print("Transcript: {}".format(result.alternatives[0].transcript))
        
    results[audio_id] = {
        'words': words,
        'transcription': transcription,
        'time': end-start
    }

ID: sources/16-01-2021_21:17:02-806145f6-ebae-43e3-848f-3d6b26d803bd. Time spent: 3.5141451358795166
ID: sources/02-02-2021_07:09:14-d866b2c0-6eca-40e1-ad93-5f4feab08421. Time spent: 2.5004448890686035
ID: sources/14-01-2021_00:30:27-39f1a1db-904a-4069-9a35-005847b9101c. Time spent: 2.379845142364502
ID: sources/03-01-2021_15:15:25-109d0932-6702-4390-9e4c-d01177e1fb5c. Time spent: 3.4720702171325684
ID: sources/15-01-2021_00:56:43-7e43bff4-5cec-48e2-8697-1e72f81a8522. Time spent: 3.633464813232422
ID: sources/03-01-2021_21:35:17-26e43698-7653-46eb-9ba4-e3a4502db126. Time spent: 2.3199291229248047
ID: sources/30-01-2021_21:49:54-c453e481-e0cc-4509-b321-f8dce76be00a. Time spent: 2.2965140342712402
ID: sources/20-01-2021_21:05:05-ab9362e1-e884-4648-86a4-d22e9cb0acf5. Time spent: 2.299475908279419
ID: sources/05-01-2021_22:50:08-fc67332b-fb3f-401b-a506-dd13e0bb3a35. Time spent: 3.060734987258911
ID: sources/23-01-2021_14:16:23-e08b689e-100e-437f-9f33-559ff3436567. Time spent: 3.67094397544

In [277]:
def get_seconds(t):
    try:
        start = t.start_time.total_seconds()
    except:
        start = 0
    
    end = t.end_time.total_seconds()
    return {"start_time": start , "end_time": end}

In [278]:
def find_word(n, words):
    if words:
        for w in words:
            if n == w.word:
                return get_seconds(w)
    else:
        return None

In [279]:
df = (
    pd.DataFrame.from_dict({
        i: {
            "time": transcription["time"],
            "transcription": transcription["transcription"],
            **{
                f"{n}_times": find_word(n, transcription["words"]) for n in VALID_SEQUENCE
            }
        }
        for i, transcription in results.items()
        # remove this id
        if i != 'sources/03-01-2021_09:52:44-109d0932-6702-4390-9e4c-d01177e1fb5c'
    }, orient='index')
    .assign(**{
        '1': lambda df: df["1_times"].notna(),
        '2': lambda df: df["2_times"].notna(),
        '3': lambda df: df["3_times"].notna(),
        '7': lambda df: df["7_times"].notna(),
        '10': lambda df: df["10_times"].notna(),
        '28': lambda df: df["28_times"].notna(),
        '29': lambda df: df["29_times"].notna(),
        '30': lambda df: df["30_times"].notna(),
    })
)

In [280]:
df[["1", "2", "3", "7", "10", "28", "29", "30"]].sum()

1     49
2     54
3     59
7     69
10    71
28    43
29    37
30    27
dtype: int64

In [301]:
df.loc[df['30']].transcription.to_list()

['1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 6 7 18 19 20 21 22 23 24 25 26 27 28 29 30 31',
 '4 5 6 7 8 9 10 11 12 13 14 15 6 7 18 19 20 21 22 23 24 25 27 28 29 30',
 '8 9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 6 7 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 y 6 7 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 6 7 8 9 20 21 22 23 24 25 26 27 28 29 30',
 '1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

In [134]:
df.time.mean()

2.1738634602776887

In [297]:
df.to_csv("/tmp/gcp_trascription_complete.csv")

In [298]:
df[["time", "1", "2", "3", "7", "10", "28", "29", "30"]].to_csv("/tmp/gcp_trascription_normalized.csv")

In [136]:
import numpy as np

In [137]:
df.shape

(87, 10)

In [138]:
df.assign(check=np.all(df[["1", "7", "10", "30"]], axis=1)).check.sum()

33

In [110]:
df.head()

,trascripcion,time,1,2,3,7,10,28,29,30
sources/16-01-2021_21:17:02-806145f6-ebae-43e3-848f-3d6b26d803bd,4 9 10 11 14 16 y 19,1.963668,True,False,False,False,False,False,False,False
sources/02-02-2021_07:09:14-d866b2c0-6eca-40e1-ad93-5f4feab08421,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 6 7,2.303372,True,True,True,True,True,False,False,False
sources/14-01-2021_00:30:27-39f1a1db-904a-4069-9a35-005847b9101c,3 4 5 6 7 8 9 10,1.967830,False,False,True,True,True,False,False,False
sources/03-01-2021_15:15:25-109d0932-6702-4390-9e4c-d01177e1fb5c,3 4 5 6 7 8 9 10 11 12 13 14 y 5,2.359141,True,True,True,True,True,False,False,False
sources/15-01-2021_00:56:43-7e43bff4-5cec-48e2-8697-1e72f81a8522,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,3.181809,True,True,True,True,True,False,False,False


In [118]:
df.trascripcion.isna().sum()

7

In [142]:
with pd.option_context("display.max_rows", 100):
    display(df)

,trascripcion,time,1,2,3,7,10,28,29,30
sources/16-01-2021_21:17:02-806145f6-ebae-43e3-848f-3d6b26d803bd,4 sitios 9 10 11 y 19,2.798251,True,False,False,False,False,False,False,False
sources/02-02-2021_07:09:14-d866b2c0-6eca-40e1-ad93-5f4feab08421,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 6 7,2.229782,True,True,True,True,True,False,False,False
sources/14-01-2021_00:30:27-39f1a1db-904a-4069-9a35-005847b9101c,34,2.147959,False,False,False,False,False,False,False,False
sources/03-01-2021_15:15:25-109d0932-6702-4390-9e4c-d01177e1fb5c,3 4 5 6 7 8 9 10 11 12 13 14 y 5,2.977011,True,True,True,True,True,False,False,False
sources/15-01-2021_00:56:43-7e43bff4-5cec-48e2-8697-1e72f81a8522,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,3.346922,True,True,True,True,True,False,False,False
sources/03-01-2021_21:35:17-26e43698-7653-46eb-9ba4-e3a4502db126,mundo 2 3 4 5 6 7 8 9 10 11 12 13 14 15 y 67,1.869251,True,True,True,True,True,False,False,False
sources/30-01-2021_21:49:54-c453e481-e0cc-4509-b321-f8dce76be00a,necesito 8 9 10 11 700 x 6779 2007 34878,1.729158,True,False,False,True,True,False,False,False
sources/20-01-2021_21:05:05-ab9362e1-e884-4648-86a4-d22e9cb0acf5,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 18 19 2...,1.811858,True,True,True,True,True,True,True,True
sources/05-01-2021_22:50:08-fc67332b-fb3f-401b-a506-dd13e0bb3a35,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 y 24,2.385878,True,True,True,True,True,False,False,False
sources/23-01-2021_14:16:23-e08b689e-100e-437f-9f33-559ff3436567,123456789 10 11 12 13 14 15 y 5,3.458901,True,True,True,False,False,False,False,False
